辛苦写了三天的代码（感觉倒像是调了3天的bug...），迭代了好几个版本，终于把 AlexNet 进行了完善，也写了 Vgg16、Vgg19、GoogLeNet，先中期小结一下吧。


先说下目前为止整体的迭代过程吧：

1. **最开始想的框架是，每个模型单独一个 py 文件，以及一个总的测试运行的 py 或者 ipynb 文件**。数据的预处理当然是统一的单独的。但是数据的处理还是花了不少时间，最终也实验了**队列+多线程**和** tf.data **两种方式，后者有一定优势，在后面的测试运行模型时都直接采用了这种方式，具体在[AlexNet及TFRocords初探.ipynb](./AlexNet及TFRocords初探.ipynb)种有比较详细记录。同时在这一步，也完成了最初 [AlexNet](./alexnet.py) 模型的搭建。

2. 但是当我继续写其他模型的时候，发现如果每个模型一个 py 文件会有大量的重复代码，为了解决 tensorflow 原生态的函数太过繁琐，之前在第一次写AlexNet模型时候便自定义了卷积、局部响应归一化、池化、全连接和写日志等辅助函数，这次在写其他模型时候，这些辅助函数能直接用上该多好，于是**决定把所有模型写进一个 py 文件，每个模型写一个类**，运行测试文件时直接调用类就行了。

3. 等写到 GoogLeNet 模型的第三个 inception 模块时，发现这样写不知道什么时候此能写完这么多 inception 模块，而且后期还得逐行检验，一个参数错了就跑不了了，网上如 conan7882 的[实现](https://github.com/conan7882/GoogLeNet-Inception-tf)虽然很完整，但是太过繁琐，而 walsvid 的[实现](https://github.com/walsvid/GoogLeNet-TensorFlow/blob/master/lib/googlenet/inception_v1.py)虽然很清晰，但是代码效率太低。最后决定自己**把inception模块也写成一个函数，**这样代码量大大减少，也便于检查。

4. 在自己跑了一遍 GoogLeNet 模型后，发现 tensorflow 里面的  ，结构非常混乱，不够清晰，之前 Vgg16、Vgg19 由于本身比较简单和直接还不觉异常，于是**把每个辅助函数都置于单独的 tf.name_scope 之下**，这样可视化的结构大大改善。

5. 这时候又发现了代码还有一个非常不 [pythonic](https://docs.python-guide.org/writing/style/) 的地方，那就是所有的调用自定义的卷积和全连接这两个辅助函数的代码行都会传递 self.REGULARIZER, self.WRITE_SUM 这两个参数，而且通常由于过长需要换行，使得本身如 Vgg16 这类结构优美的模型完全没有从代码种体现出来，于是想着如何砍掉这两个参数。首先想到的当时是直接将二者写为固定的常量，但是由于还是想保留一些整体的灵活性，留住这两个参数，于是转而想其他的方法。二者是自定义类的属性，不能调用的根本是因为辅助函数不在自定义类里面啊，于是准备**将辅助函数全部写为自定义类的方法**，这样就可以直接调用，不用到处传递了。

6. 但是将辅助函数写进一个模型的类还好，写第二个时候，我就不愿意了，又有大量重复代码，基本相当于每个模型一个类，每个类一个 py 文件了，又不能复用了，于是想着如何这些辅助函数也能复用，这就有了这个思路：**直接写一个 CNNs 的类，在里面定义一些公共的属性和方法，然后将每个模型分别写作它的一个方法**。

7. 这个时候的模型定义文件已经比较简洁了，建立模型就是这个样子:

   `model = CNNs(x, NUM_CLASSES, keep_prob, regularizer, WRITE_LOG)
   alexnet = model.alexnet()
   `
   
   看起来还不错，但是等到调用前向传播的时候就比较别扭了：`y = model.last`，而不是`y = alexnet.last`，这一点看起来实在难以接受，而且 model 会有大量的属性重复，否则需要复杂的命名。于是继续想其他办法。既然我是想调用`alexnet.last`这种，那么 last 就必定是属于 alexnet 的方法，这样的 alexnet 看起来又像是一个类了，于是最后想到了最终的方案：**将每个模型从CNNs类中的方法提出来变成一个个单独的类，这样依然可以继承CNNs中的属性和方法，每个子类也可以定义一些自己的属性和方法**，比如 GoogLeNet 的辅助分支。
   
说完迭代，再说点其他：
1. 当然整个过程中还有其他细节的优化，比如：
   - 对于卷积层和全连接层，定义权重时需要知道输入数据的 channels 或者 nodes ，但是这两个参数可以通过直接计算本层输入获得，不用自己单独制定，少一个参数就少一份出 bug 的几率；
   - 将模型类的构建方法中的所有函数都自定义一遍，包括简单的 dropout 和 concat 等，这样便于代码的一致性和可视化的效果；
   - name_scope 的命名和结构也是提升整体美观的一部分。
   
2. 但是整个框架的优化还是有一定的保留和取舍，比如：CNNs 下面的公共方法中，卷积核的长宽、步长的长宽，虽然在目前模型中它们都是各自相等的，只定义一个可以节省不少参数，但是为了保留后期的灵活性，还是保留了。

3. 另外也还有一些待优化的地方吧，比如如果要建立多个模型，那么在初始化`super().__init__`的时候可能在父类处有一些冲突，再比如 CNNs 下面的卷积和全连接方法，激活函数都是默认的 relu，也没有留出选择的余地，目前还是考虑到当前的需要和代码的简洁。